# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [40]:
# you must populate this dict with the schools required -> try talking to the teaching team about this


schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035    
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests
import getpass


def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])
  
    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [41]:
# could you write this as a list comprehension? ;)
#comments = []
#for school in schools.keys():
#    print(school)
#    comments.append(get_comments_school(school))

comments = [get_comments_school(school) for school in schools.keys()]

comments = pd.concat(comments)

In [ ]:
"""olhando o todo da tabela"""
comments.describe

In [23]:
#comments.columns
stud_a = comments.drop(columns = ['hostProgramName','body','user','comments','review_body'])
#stud_a = comments._get_numeric_data()
stud_a.describe
#stud_a.head()
stud_a.isnull().sum()


id                   0
name                 0
anonymous            0
graduatingYear       4
isAlumni             0
jobTitle          1428
tagline              1
createdAt            0
queryDate            0
program            179
overallScore         8
overall             10
curriculum          47
jobSupport         247
school               0
dtype: int64

In [24]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



ironhack
app-academy
springboard


In [36]:
locations_list
locations = pd.concat(locations_list)

#print(locations.shape)
locations.isnull().sum()
"""As únicas colunas completas são: id, description, school, school_id"""
locations.describe

locations_a = locations.drop(columns=['country.id','country.name','country.abbrev','city.id','city.name','city.keyword', 'state.id', 'state.name','state.abbrev','state.keyword'])
locations_a.head()

#local = locations_a['description'].tolist()
#print(local)

#city_name = []
#country_name = []

#local_lst = []
#for line in local:
#    for item in line.split(','):
#        #local_lst.append(item)
#        if item != 'Online':
#            city_name.append(item)
#            country_name.append(item)
#        else:
#            city_name.append('Online')
#            country_name.append('Online')
#city_name

locations_a['description'] = locations_a['description'].apply(lambda x: re.sub('Online', 'Online, Online',x))
locations_a['city_name'] = locations_a['description'].apply(lambda x: x.split(',')[0])
locations_a['country_name'] = locations_a['description'].apply(lambda x: x.split(',')[1])
locations_a.drop(columns='description',inplace=True)
locations_a.rename(columns={'id':'location_id'},inplace=True)
locations_a.head()

,location_id,school,school_id,city_name,country_name
0,15901,ironhack,10828,Berlin,Germany
1,16022,ironhack,10828,Mexico City,Mexico
2,16086,ironhack,10828,Amsterdam,Netherlands
3,16088,ironhack,10828,Sao Paulo,Brazil
4,16109,ironhack,10828,Paris,France


In [45]:
courses = pd.concat(courses_list)
courses['courses'].unique()
#courses['course_area']
#courses['course_type']

array(['Data Analytics Bootcamp', 'Data Analytics Part-Time',
       'UX/UI Design Bootcamp', 'UX/UI Design Part-Time',
       'Web Development Bootcamp', 'Web Development Part-Time',
       'Cyber Security Bootcamp', 'Bootcamp Prep',
       'Software Engineer Track: In-Person',
       'Software Engineer Track: Online', 'Data Analytics Career Track',
       'Data Engineering Career Track', 'Data Science Career Track',
       'Data Science Career Track Prep', 'Digital Marketing Career Track',
       'Introduction to Cybersecurity', 'Introduction to Design',
       'Machine Learning Engineering Career Track',
       'Software Engineering Career Track',
       'Software Engineering Career Track Prep Course',
       'UI/UX Design Career Track', 'UX Career Track'], dtype=object)

In [10]:
badges = pd.concat(badges_list)
badges.head()

,description,keyword,name,school,school_id
0,<p>School offers fully online courses</p>,available_online,Available Online,ironhack,10828
1,<p>School publishes a third-party verified out...,verified_outcomes,Verified Outcomes,ironhack,10828
2,<p>School offers part-time and evening classes...,flexible_classes,Flexible Classes,ironhack,10828
0,<p>School offers fully online courses</p>,available_online,Available Online,app-academy,10525
1,<p>School offers part-time and evening classes...,flexible_classes,Flexible Classes,app-academy,10525


In [11]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools.head()

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>App Academy is a ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035
